In [1]:
# 필요한 라이브러리 임포트
import sqlite3
from datetime import datetime
import pandas as pd


# 데이터베이스 설정
DB_PATH = "database/sqlite_cache.db"
SQLITE_TABLE_NAME = "AI_COMMENTS"

# 전체 데이터 조회 함수
def view_all_cache():
    with sqlite3.connect(DB_PATH) as conn:
        query = f"""
        SELECT id, company_name, analysis_type, analysis_metric, 
               detailed_result, summary, created_at
        FROM {SQLITE_TABLE_NAME}
        ORDER BY id DESC, analysis_type
        """
        df = pd.read_sql_query(query, conn)
        return df

# 특정 회사의 데이터 조회 함수
def view_company_cache(company_name: str):
    with sqlite3.connect(DB_PATH) as conn:
        query = f"""
        SELECT id, company_name, analysis_type, analysis_metric, 
               detailed_result, summary, created_at
        FROM {SQLITE_TABLE_NAME}
        WHERE company_name = ?
        ORDER BY id DESC, analysis_type
        """
        df = pd.read_sql_query(query, conn, params=[company_name])
        return df

# ID로 특정 레코드 삭제 함수
def delete_by_id(record_id: int):
    with sqlite3.connect(DB_PATH) as conn:
        cursor = conn.cursor()
        query = f"""
        DELETE FROM {SQLITE_TABLE_NAME}
        WHERE id = ?
        """
        cursor.execute(query, [record_id])
        deleted_count = cursor.rowcount
        conn.commit()
        return deleted_count

# 여러 ID로 레코드 삭제 함수
def delete_multiple_by_ids(record_ids: list):
    with sqlite3.connect(DB_PATH) as conn:
        cursor = conn.cursor()
        placeholders = ','.join('?' * len(record_ids))
        query = f"""
        DELETE FROM {SQLITE_TABLE_NAME}
        WHERE id IN ({placeholders})
        """
        cursor.execute(query, record_ids)
        deleted_count = cursor.rowcount
        conn.commit()
        return deleted_count
    
def reset_table_ids():
    with sqlite3.connect(DB_PATH) as conn:
        # 1. 테이블을 DataFrame으로 읽기
        df = pd.read_sql_query(f"SELECT * FROM {SQLITE_TABLE_NAME}", conn)
        
        # 2. 인덱스 리셋
        df = df.reset_index(drop=True)
        df.index += 1  # 1부터 시작하도록 설정
        
        # 3. id 컬럼 업데이트
        df['id'] = df.index
        
        # 4. 기존 테이블 삭제 후 재생성
        df.to_sql(SQLITE_TABLE_NAME, conn, if_exists='replace', index=False)
        
        
def reset_primary_key():
    with sqlite3.connect(DB_PATH) as conn:
        cursor = conn.cursor()
        
        # 1. 기존 테이블 이름 변경
        cursor.execute(f"ALTER TABLE {SQLITE_TABLE_NAME} RENAME TO temp_table;")
        
        # 2. 새 테이블 생성 (id를 PRIMARY KEY로 설정)
        cursor.execute(f"""
            CREATE TABLE {SQLITE_TABLE_NAME} (
                id INTEGER PRIMARY KEY,
                company_name TEXT,
                analysis_type TEXT,
                analysis_metric TEXT,
                detailed_result TEXT,
                summary TEXT,
                created_at TIMESTAMP
            );
        """)
        
        # 3. 데이터 복사
        cursor.execute(f"""
            INSERT INTO {SQLITE_TABLE_NAME} 
            SELECT * FROM temp_table;
        """)
        
        # 4. 임시 테이블 삭제
        cursor.execute("DROP TABLE temp_table;")
        
        conn.commit()

In [4]:
# 전체 캐시 데이터 조회
df_all = view_all_cache()
display(df_all)

df_all.to_csv("sample_db.csv", encoding='utf-8-sig', index=None)


,id,company_name,analysis_type,analysis_metric,detailed_result,summary,created_at
0,31,Company_2,insight,financial_stability/profitability,None,회사의 재무 안정성은 높은 유동비율을 바탕으로 견고하게 유지되고 있습니다. `수익성...,2024-12-05 04:57:23
1,30,Company_2,insight,financial_stability/partner_stability,None,"회사 재정 안정성은 개선되고 있으며, 2023년에 현재 비율이 177.13%로 업종...",2024-12-05 04:45:02
2,29,Company_2,strength,financial_stability,The company's financial stability shows promis...,당사의 유동비율은 2022년 73.64%에서 2023년 177.13%로 크게 개선되...,2024-12-05 04:44:58
3,28,Company_1,insight,financial_stability/partner_stability,None,회사의 재무 안정성은 유동비율이 2023년 94.65%에서 2024년 115.75%...,2024-12-05 04:36:24
4,27,Company_1,strength,financial_stability,The company's financial stability shows promis...,당사의 유동비율은 2023년 94.65%에서 2024년 115.75%로 개선될 것으...,2024-12-05 04:36:21
5,26,Company_3,insight,growth/financial_stability,None,회사는 2023년 수익률 49.77% 증가 및 자산 관리 능력을 통해 타 산업 대비...,2024-12-04 05:20:42
6,25,Company_3,weakness,financial_stability,The company's financial stability shows concer...,"당사의 단기 차입금은 2021년 900백만 원에서 2024년 1,691백만 원으로 ...",2024-12-04 05:20:39
7,24,Company_3,strength,growth,The company's financial performance shows a mi...,"당사의 2023년 연간 수익은 69,985백만 원으로, 전년 대비 49.77% 증가...",2024-12-04 05:20:31
8,23,Company_3,insight,profitability/growth,None,"회사의 영업이익과 당기순이익의 회복은 인상적이며, 2024년 예상 영업이익이 408...",2024-12-03 08:21:51
9,22,Company_3,weakness,growth,The company's financial performance reveals se...,"당사의 2023년 매출액은 69,985백만 원으로, 2022년 46,728백만 원에...",2024-12-03 08:21:47


In [118]:
# 특정 회사 데이터 조회
company_df = view_company_cache("Company_1")
display(company_df)


,id,company_name,analysis_type,analysis_metric,detailed_result,summary,created_at
0,16,Company_1,insight,growth/profitability,None,회사의 연간 매출액은 2023년 62406백만 원으로 전년대비 대폭 성장을 기록하며...,2024-12-03 08:03:07
1,15,Company_1,weakness,profitability,The analysis of the company's profitability re...,당사의 연간 영업이익은 2021년 -2874백만 원에서 2023년 1480백만 원으...,2024-12-03 08:03:04
2,14,Company_1,insight,profitability/partner_stability,None,회사의 2023년 수익성은 역전 극복의 강점을 보이며 운영 이익은 영업손실 2874...,2024-12-03 07:02:33
3,13,Company_1,strength,profitability,The analysis of the company's profitability re...,당사는 2023년 영업이익 1480백만 원을 기록하며 전년 대비 2874백만 원의 ...,2024-12-03 07:02:29
4,12,Company_1,insight,growth/partner_stability,None,"회사의 강점은 `특히 매출 성장 면에서 두드러지며`, 기업은 2023년에 전년 대비...",2024-12-03 06:26:05
5,11,Company_1,weakness,partner_stability,The analysis of the company's business partner...,"당사의 주요 매출처 5곳이 전체 연매출의 68.9%인 225,075백만 원을 차지하...",2024-12-03 06:26:00
6,10,Company_1,strength,growth,The company's financial performance shows a mi...,"당사의 2023년 연간 수익은 62,406백만 원으로, 전년 대비 27.04% 감소...",2024-12-03 06:25:49


In [108]:
# # 특정 ID의 레코드 삭제
# deleted = delete_by_id(10)
# print(f"{deleted}개의 레코드가 삭제되었습니다.")

# 여러 ID의 레코드 삭제
ids_to_delete = [10, 11]
deleted = delete_multiple_by_ids(ids_to_delete)
print(f"{deleted}개의 레코드가 삭제되었습니다.")

2개의 레코드가 삭제되었습니다.


In [76]:
print(sqlite3.sqlite_version)

3.45.1


In [101]:
reset_table_ids()

In [106]:
reset_primary_key()

In [9]:
DB_CREDIT_PATH = "database/sqlite_credit_cache.db"
conn = sqlite3.connect(DB_CREDIT_PATH)
cursor = conn.cursor()

try:
    # 테이블의 모든 데이터 삭제
    cursor.execute("DELETE FROM CREDIT_CONSULTING")
    
    # 변경사항 저장
    conn.commit()
    print("Successfully deleted all records from CREDIT_CONSULTING table")
    
except sqlite3.Error as e:
    print(f"An error occurred: {e}")
    
finally:
    # 연결 종료
    conn.close()

Successfully deleted all records from CREDIT_CONSULTING table
